# Visualize

## WBS

1. create an xarray using dask for all annual etasw outputs from a chip



In [ ]:
import xarray as xr
import hvplot.xarray
import dask
from dask.distributed import Client, progress
#from dask_kubernetes import KubeCluster
import os
import fsspec

In [ ]:
from pangeoLib.aws_authenticate import aws_authenticate

aws_authenticate()

In [ ]:
fs = fsspec.filesystem('s3', anon=False, requester_pays=True)

In [ ]:
file_objects = fs.ls('dev-et-data/tiles/')
for file_obj in file_objects:
    if '32' in file_obj:
        print(file_obj)

In [ ]:
chip='dev-et-data/tiles/tile40N-90E_chip32N-84E'

In [ ]:
def get_etasw_tifs(bucket_prefix_path):
    files = fs.ls(bucket_prefix_path)
    tifs = [f for f in files if f.endswith('tif')]
    etasw_list = [f for f in tifs if 'etasw_' in f]
    return(etasw_list)

In [ ]:
ets = get_etasw_tifs(chip)

In [ ]:

years = ['2013', '2014', '2015']

et_annual_tifs = []
for year in years:
    et_tif = f'{chip}/etasw_{year}.tif'
    print(et_tif)
    et_annual_tifs.append(et_tif)



In [ ]:
os.environ["AWS_REQUEST_PAYER"] = "requester" 

client = Client()
client

In [ ]:
@dask.delayed
def tif_to_da(tif):
    return xr.open_rasterio('s3://'+tif, chunks={'band':1, 'x':512, 'y':512})

In [ ]:
%%time
tifs = et_annual_tifs
lazy_da =[tif_to_da(tif) for tif in tifs]
dalist = dask.compute(*lazy_da)
da = xr.concat(dalist, dim='band')
da = da.rename({'band':'year'})

In [ ]:
!aws s3 ls --request-payer requester dev-et-data/tiles/tile40N-90E_chip32N-84E/etasw_2016.tif

In [ ]:
da

In [ ]:
da = da.assign_coords(year=range(2013,2016))

In [ ]:
da

In [ ]:
ds = da.to_dataset(name='etasw')

In [ ]:
ds

In [ ]:
%%time
ds.etasw[1,:,:].load().hvplot(grid=True)

In [ ]:
%%time
ds.etasw[1,:,:].load().hvplot(grid=True, width=962, height=962, cmap='viridis')

In [ ]:
#dev-et-data/tiles/tile40N-90E_chip32N-84E/etasw_2016.tif

In [ ]:
import rasterio
from shapely.geometry import box
file='/vsis3/dev-et-data/tiles/tile40N-90E_chip32N-84E/etasw_2015.tif'
src = rasterio.open(file)
src.shape
row=src.bounds
my_useful_geometry = box(row.left, row.bottom, row.right, row.top)

In [ ]:
import folium
import geopandas
map_osm = folium.Map(location=[30, -82], zoom_start=7)

style_function = lambda x: {'fillColor': 'red', 'color': 'green'} 


my_useful_geojson = geopandas.GeoSeries([my_useful_geometry]).to_json()
gjson = folium.GeoJson(my_useful_geojson, style_function=style_function, tooltip=file).add_to(map_osm)

map_osm

In [ ]:
ds.sel(year=2013).etasw.load().hvplot(grid=True, width=962, height=962, cmap='viridis')

In [ ]:
ds.sel(year=2015).etasw.load().hvplot(grid=True, width=962, height=962, cmap='viridis')